# Steady state solution

In [43]:
using gmsh

using LinearAlgebra, SparseArrays
using WriteVTK

using BenchmarkTools

include("../constants.jl")
include("../get_mesh_data.jl")
include("../process.jl")
include("../definitions/general.jl")
include("../definitions/assemble_Kf.jl")

const MESH_LOCATION = "../../mesh/transformer_stedin.msh"
const OUTPUT_LOCATION = "../../out/"

using Logging

In [44]:
gmsh.open(MESH_LOCATION)
mshdata = get_mesh_data();

Info    : Reading '../../mesh/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../../mesh/transformer_stedin.msh'


## Problem definition

Find $A_z$ in the system
$$ -\nabla \times \left[\frac{1}{\mu}\nabla A_z\right] = \mathbf J_0, $$
where
- $A_z$ is the current density in the $z$ direction.
- $\mu$ is the permeability of the core.
- $\mathbf J_0$ is the imposed source current density.
- $\sigma$: the conductivity of the material (not used).

### Parameters

In [45]:
"Primary peak phase current"
Ip = 0;

"Secondary peak phase current"
Is = 777.62;

Np = 266;
Ns = 6;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Vacuum permeability
μ_0 = 4e-7 * pi;

# Relative permeability of the core
μ_r = 1000;    

### Source, linear reluctivity per element (no conductivity due to steady state)


In [46]:
source_per_element = map(
    id -> source(Jp, Js, id),
    mshdata.e_group
);

reluctivity_per_element = map(
    id -> linear_reluctivity(μ_0, μ_r, id),
    mshdata.e_group
);

conductivity_per_element = map(
    id -> 0,
    mshdata.e_group
);

## System of equations

In [56]:
@timev K, f = assemble_Kf(mshdata,source_per_element,reluctivity_per_element);

# stop printing stuff bro
nothing

  0.126435 seconds (1.51 M allocations: 115.180 MiB, 6.54% gc time)
elapsed time (ns):  126435400
gc time (ns):       8274100
bytes allocated:    120775392
pool allocs:        1512049
non-pool GC allocs: 2
malloc() calls:     14
realloc() calls:    1
free() calls:       15
minor collections:  2
full collections:   0


How long does this take?

In [39]:
# @timev K, f = assemble_Kf(mshdata, source_per_element, reluctivity_per_element);

MethodError: MethodError: no method matching Vector{Int64}(::typeof(zero), ::Int64)
Closest candidates are:
  Vector{T}(!Matched::UndefInitializer, ::Int64) where T at boot.jl:459
  Array{T, N}(!Matched::UndefInitializer, ::Int64...) where {T, N} at boot.jl:465
  Vector{T}(!Matched::UndefInitializer, ::Integer) where T at baseext.jl:23
  ...

Solve the system.

In [28]:
u = K \ f;

## Post

In [15]:
# Post-process for magnetic field and current density
omega = 0
B, H, Wm, Jel = process(mshdata, u, source_per_element, reluctivity_per_element, conductivity_per_element, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

Save the result

In [16]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid(string(OUTPUT_LOCATION, "transformer9.vtu"), points, cells);


# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["B_vec", VTKCellData()] = real.(B);
vtkfile["Jel", VTKCellData()]   = Jel;


# Save the file
print("Saving result in a file...")
outfiles = vtk_save(vtkfile);
println(" Done.")

Saving result in a file... Done.
